In [ ]:
!pip install -q sentence-transformers transformers datasets torch faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
## mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 1: Load JSON File. The preprocessed CUAD dataset was saved

In [ ]:
import json

# Load preprocessed CUAD data
with open("/content/drive/My Drive/Colab Notebooks/NLP_266_Project/CUAD_preprocessed_data.json", "r") as f:
    dataset= json.load(f)


In [ ]:
# Check dataset structure
print(f"Total Documents: {len(dataset)}")
print(f"Sample Document: {json.dumps(dataset[5], indent=4)}")


Total Documents: 20910
Sample Document: {
    "contract_title": "LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT",
    "input": "Extract the Renewal Term from this document: EXHIBIT 10.6 DISTRIBUTOR AGREEMENT THIS DISTRIBUTOR AGREEMENT (the \"Agreement\") is made by and between Electric City Corp., a Delaware corporation (\"Company\") and Electric City of Illinois LLC (\"Distributor\") this 7th day of September, 1999. RECITALS A. The Company's Business. The Company is presently engaged in the business of selling an energy efficiency device, which is referred to as an \"Energy Saver\" which may be improved or otherwise changed from its present composition (the \"Products\"). The Company may engage in the business of selling other products or other devices other than the Products, which will be considered Products if Distributor exercises its options pursuant to Section 7 hereof. B. Representations. As an inducement to the Company to enter into this Agreement, the Distributor has rep

Is the Dataset Suitable for Retrieval?
We now test SBERT-based hierarchical retrieval to determine:

Can we extract relevant sections from contracts efficiently?
Can retrieval alone answer clause-based queries, or is chunking required?

Hierarchical Retrieval using SBERT
We use Sentence-BERT (SBERT) to embed contract texts and retrieve relevant sections.

computed SBERT embeddings for all document sections and stored them efficiently, the next steps are:

Run the optimized retrieval function to extract relevant sections.
Pass the best-matched section to FLAN-T5 for final clause extraction.
Evaluate the results and iterate if needed.

In [ ]:
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Load SBERT model and move it to GPU
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2')
device = "cuda" if torch.cuda.is_available() else "cpu"
retrieval_model = retrieval_model.to(device)

# Function to split contract text into smaller sections
def split_into_sections(contract_text):
    return contract_text.split(". ")  # Split into meaningful chunks

# Step 1: Process document texts (split into sections)
print("🔄 Splitting documents into sections...")
document_texts = [split_into_sections(doc["input"]) for doc in dataset]

# Step 2: Compute embeddings in batches with GPU acceleration
print("⚡ Computing SBERT embeddings for document sections...")
document_embeddings = []

# Flatten sections for batch encoding
flat_sections = [section for doc in document_texts for section in doc]
batch_size = 32  # Adjust batch size based on GPU memory

with torch.no_grad():  # Disable gradient tracking for inference
    for i in tqdm(range(0, len(flat_sections), batch_size), desc="🔄 Encoding batches"):
        batch = flat_sections[i : i + batch_size]
        batch_embeddings = retrieval_model.encode(batch, convert_to_tensor=True, device=device)
        document_embeddings.extend(batch_embeddings)

# Reshape embeddings back to document structure
structured_embeddings = []
idx = 0
for doc_sections in document_texts:
    num_sections = len(doc_sections)
    structured_embeddings.append(document_embeddings[idx : idx + num_sections])
    idx += num_sections

# Final document embeddings are structured per document
document_embeddings = structured_embeddings

print("✅ Document embeddings computed and structured successfully!")


🔄 Splitting documents into sections...
⚡ Computing SBERT embeddings for document sections...


🔄 Encoding batches: 100%|██████████| 159829/159829 [35:37<00:00, 74.76it/s]


✅ Document embeddings computed and structured successfully!


Next Steps to Test Retrieval and Ensure it Works Correctly
Once contract documents are embedded and optimized for retrieval, we now need to verify that retrieval is working properly before passing it to FLAN-T5 for answer generation.

🔹 Step 1: Test Retrieval for a Few Sample Queries
We need to run sample queries and check whether the most relevant sections are retrieved.

In [ ]:
import torch
from sentence_transformers import util
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load FLAN-T5 (for clause extraction)
flan_model_name = "google/flan-t5-base"  # Change to "flan-t5-large" for better accuracy
flan_tokenizer = AutoTokenizer.from_pretrained(flan_model_name)
flan_model = AutoModelForSeq2SeqLM.from_pretrained(flan_model_name).to(device)

def retrieve_and_extract(query, contract_title, document_texts, document_embeddings, document_titles, top_k=3):
    """
    Retrieves the most relevant sections for a given query and refines it using FLAN-T5.
    Prioritizes extracted sections containing numerical values (e.g., contract durations).
    """

    # Step 1: Find the matching document index
    matching_docs = [idx for idx, title in enumerate(document_titles) if contract_title in title]

    if not matching_docs:
        print(f"⚠️ No matching document found for: {contract_title}")
        return None

    # Step 2: Extract the document sections
    doc_index = matching_docs[0]  # Assume one matching document for now
    relevant_sections = document_texts[doc_index]  # Sections of the document
    section_embeddings = torch.stack(document_embeddings[doc_index])  # Convert list to tensor

    # Step 3: Compute similarity scores
    query_embedding = retrieval_model.encode(query, convert_to_tensor=True, device=device)
    similarities = util.pytorch_cos_sim(query_embedding, section_embeddings)

    # Step 4: Retrieve top-k most relevant sections
    similarities = similarities.squeeze()  # Ensure 1D tensor
    top_indices = similarities.topk(top_k, dim=-1).indices.tolist()  # Fix dimension issue
    retrieved_sections = [relevant_sections[idx] for idx in top_indices]

    print(f"\n🔍 Retrieved Sections:\n")
    for i, section in enumerate(retrieved_sections):
        print(f"🔹 Section {i+1}: {section}\n")

    # Step 5: Post-processing - Prioritize sections containing numbers (e.g., years, durations)
    def count_numbers(text):
        return sum(c.isdigit() for c in text)

    best_match = max(retrieved_sections, key=count_numbers)  # Prefer section with numbers

    print(f"\n✅ Best Matched Section for Extraction:\n{best_match}\n")

    # Step 6: Pass the best section to FLAN-T5 for refined extraction
    flan_input = f"Extract the complete answer for: {query} {best_match}"
    inputs = flan_tokenizer(flan_input, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = flan_model.generate(**inputs, max_length=128)

    extracted_answer = flan_tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"\n🎯 FLAN-T5 Extracted Answer:\n{extracted_answer}\n")
    return extracted_answer


# Step 1: Run retrieval and extraction
test_query = "Extract the Renewal Term from this document:"
test_contract_title = "LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT"

retrieved_answer = retrieve_and_extract(
    test_query,
    test_contract_title,
    document_texts,
    document_embeddings,
    document_titles
)

print(f"\n🔎 **Final Extracted Answer:** {retrieved_answer}")



🔍 Retrieved Sections:

🔹 Section 1: All renewals of this Agreement shall be on the same terms and conditions as are set forth herein

🔹 Section 2: The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter

🔹 Section 3: (A) 375 units in the first Product Year (1999) (B) 750 units in the next succeeding Product Year; (2000) (C) 937 units in the next succeeding Product Year; (2001) (D) 1,171 units in the next succeeding Product Year; (2002) (E) 1,463 units in the next succeeding Product Year; (2003) (F) 1,828 units in the next succeeding Product Year; (2004) (G) 2,285 units in the next succeeding Product Year; (2005) (H) 2,856 unit each in the lat three years of the initial Term of this Agreement and any renewals thereof


✅ Best Matched Section for Extraction:
(A) 375 units in the first Product Year (1999) (B) 750 units in the next succeeding Product Year; (2000)

In [ ]:
pip install -q rank_bm25

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# ✅ Load Retrieval Models (Efficient Dense + Sparse Retrieval)
bm25_corpus = [" ".join(doc) for doc in document_texts]  # Flatten sections for BM25
bm25_tokenized = [doc.split() for doc in bm25_corpus]
bm25 = BM25Okapi(bm25_tokenized)

# ✅ Load SBERT for Contrastive Reranking
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2').to("cuda")

# ✅ Load FLAN-T5 for Clause Extraction
flan_model_name = "google/flan-t5-base"
flan_tokenizer = AutoTokenizer.from_pretrained(flan_model_name)
flan_model = AutoModelForSeq2SeqLM.from_pretrained(flan_model_name).to("cuda")



In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util

# ✅ Load SBERT model
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2').to("cuda")

def sbert_retrieve(query, contract_title, document_texts, document_embeddings, document_titles, top_k=3):
    """
    Retrieves the most relevant sections using only SBERT, avoiding BM25 indexing issues.
    """

    # ✅ Step 1: Find the matching document
    matching_docs = [idx for idx, title in enumerate(document_titles) if contract_title in title]
    if not matching_docs:
        print(f"⚠️ No matching document found for: {contract_title}")
        return None

    doc_index = matching_docs[0]  # Assume one matching document
    relevant_sections = document_texts[doc_index]  # Sections of the document
    section_embeddings = document_embeddings[doc_index]  # Embeddings of sections

    # ✅ Ensure document contains sections
    if len(relevant_sections) == 0:
        print(f"⚠️ No sections found in document {contract_title}")
        return None

    # ✅ Step 2: Compute SBERT Similarity Scores
    query_embedding = retrieval_model.encode(query, convert_to_tensor=True, device="cuda")
    similarities = util.pytorch_cos_sim(query_embedding, torch.stack(section_embeddings))  # Compute similarity scores

    # ✅ Ensure safe indexing by restricting to available sections
    top_indices = similarities.squeeze().topk(min(top_k, len(relevant_sections))).indices.tolist()
    retrieved_sections = [relevant_sections[idx] for idx in top_indices]

    print(f"\n🔍 Retrieved Sections (SBERT Only):\n")
    for i, section in enumerate(retrieved_sections):
        print(f"🔹 Section {i+1}: {section}\n")

    # ✅ Return Best Section
    return retrieved_sections[0] if retrieved_sections else None

# ✅ Test the SBERT-Only Retrieval
test_queries = [
    "Extract the Renewal Term from this document:",
    "What is the Governing Law for this contract?",
    "List the Payment Terms mentioned in this agreement.",
    "What are the Termination conditions?"
]

test_contract_title = "LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT"

for query in test_queries:
    retrieved_section = sbert_retrieve(
        query,
        test_contract_title,
        document_texts,
        document_embeddings,
        document_titles
    )

    print(f"\n🔎 **Final Retrieved Section for '{query}':** {retrieved_section}\n")



🔍 Retrieved Sections (SBERT Only):

🔹 Section 1: All renewals of this Agreement shall be on the same terms and conditions as are set forth herein

🔹 Section 2: The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter

🔹 Section 3: (A) 375 units in the first Product Year (1999) (B) 750 units in the next succeeding Product Year; (2000) (C) 937 units in the next succeeding Product Year; (2001) (D) 1,171 units in the next succeeding Product Year; (2002) (E) 1,463 units in the next succeeding Product Year; (2003) (F) 1,828 units in the next succeeding Product Year; (2004) (G) 2,285 units in the next succeeding Product Year; (2005) (H) 2,856 unit each in the lat three years of the initial Term of this Agreement and any renewals thereof


🔎 **Final Retrieved Section for 'Extract the Renewal Term from this document:':** All renewals of this Agreement shall be on the s

3.3 Retrieve Relevant Sections - Document-Specific Retrieval Function
We query the retrieval model with the clause-based question.

If using SBERT embeddings, modify the function so it:

Filters only the relevant document first (using its title or ID).
Then retrieves the most relevant section within that document.

In [ ]:
import torch
from tqdm import tqdm
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

# Load DPR Context Encoder
dpr_context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to("cuda")
dpr_context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

def compute_dpr_embeddings(doc_texts):
    """
    Compute DPR embeddings for all sections of all documents.
    Ensures all embeddings are stored in (num_sections, 768) format.
    """
    document_embeddings = []

    with torch.no_grad():
        for doc_sections in tqdm(doc_texts, desc="⚡ Encoding Documents with DPR"):
            section_embeddings = []
            for section in doc_sections:
                encoded = dpr_context_tokenizer(section, return_tensors="pt", truncation=True, max_length=512).to("cuda")
                embedding = dpr_context_encoder(**encoded).pooler_output  # Extract vector (1, 768)
                section_embeddings.append(embedding.squeeze(0))  # Ensure shape is (768,)

            document_embeddings.append(torch.stack(section_embeddings))  # Stack into (num_sections, 768)

    return document_embeddings

# ✅ Compute DPR embeddings (ensure correct shape)
document_embeddings = compute_dpr_embeddings(document_texts)


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

KeyboardInterrupt: 

In [ ]:
def dpr_retrieve(query, contract_title, document_texts, document_embeddings, document_titles, top_k=3):
    """
    Uses DPR for passage retrieval, then SBERT for re-ranking.
    """

    # ✅ Step 1: Find Matching Document
    matching_docs = [idx for idx, title in enumerate(document_titles) if contract_title in title]
    if not matching_docs:
        print(f"⚠️ No matching document found for: {contract_title}")
        return None

    doc_index = matching_docs[0]  # Assume one matching document
    relevant_sections = document_texts[doc_index]  # Get document sections
    section_embeddings = document_embeddings[doc_index]  # Get stored embeddings

    # ✅ Step 2: Compute DPR Query Embedding
    query_tokens = dpr_query_tokenizer(query, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    query_embedding = dpr_query_encoder(**query_tokens).pooler_output  # Extract vector

    # ✅ Step 3: Compute DPR Similarities
    section_tensor = torch.stack(section_embeddings)  # Convert list to tensor
    dpr_similarities = torch.matmul(query_embedding, section_tensor.T).squeeze()  # Cosine similarity

    # ✅ Step 4: Retrieve Top-k Candidates
    top_indices = dpr_similarities.topk(min(top_k, len(relevant_sections))).indices.tolist()
    retrieved_sections = [relevant_sections[idx] for idx in top_indices]

    print(f"\n🔍 Retrieved Sections (DPR):\n")
    for i, section in enumerate(retrieved_sections):
        print(f"🔹 Section {i+1}: {section}\n")

    # ✅ Step 5: Re-Rank with SBERT for Higher Accuracy
    sbert_query_embedding = sbert_model.encode(query, convert_to_tensor=True, device="cuda")
    section_embeddings_sbert = sbert_model.encode(retrieved_sections, convert_to_tensor=True, device="cuda")
    sbert_similarities = util.pytorch_cos_sim(sbert_query_embedding, section_embeddings_sbert)

    # ✅ Step 6: Select Best Section
    best_idx = sbert_similarities.argmax().item()
    best_section = retrieved_sections[best_idx]

    print(f"\n✅ Best Matched Section:\n{best_section}\n")

    return best_section


# ✅ Run DPR-based Retrieval
test_queries = [
    "Extract the Renewal Term from this document:",
    "What is the Governing Law for this contract?",
    "List the Payment Terms mentioned in this agreement.",
    "What are the Termination conditions?"
]

test_contract_title = "LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT"

for query in test_queries:
    retrieved_section = dpr_retrieve(
        query,
        test_contract_title,
        document_texts,
        document_embeddings,
        document_titles
    )

    print(f"\n🔎 **Final Retrieved Section for '{query}':** {retrieved_section}\n")
